# Satellite Humidity Profiles: Variability of vertically resolved Tropospheric Humidity

## Import packages

In [ ]:
import calendar

import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
start = "2006-01"
stop = "2023-02"

# Product type
product_type = "reanalysis_data"
assert product_type in ("reanalysis_data", "radio_occultation_data")

# Altitude bins
bins = (0, 1_000, 2_000, 4_000, 6_000, 12_000)

## Define Request

In [ ]:
collection_id = "satellite-humidity-profiles"
request = {
    "variable": "all",
    "product_type": product_type,
}
request = download.update_request_date(
    request, start=start, stop=stop, stringify_dates=True
)

## Download Data

In [ ]:
ds = download.download_and_transform(collection_id, request, chunks={"year": 1})
ds = (
    ds.squeeze(drop=True)
    .set_coords([var for var, da in ds.data_vars.items() if set(da.dims) == {"time"}])
    .compute()
)
ds_monthly = ds.groupby("month").mean()

## Plot seasonal cycle

In [ ]:
da = ds_monthly["Q"].assign_coords(
    month=[calendar.month_abbr[month] for month in ds_monthly["month"].values]
)

plot_kwargs = {"x": "month", "levels": 9}
for alt_start, alt_stop in zip(bins[:-1], bins[1:]):
    title = f"{da['alt'].attrs['long_name']}: {alt_start}-{alt_stop} {da['alt'].attrs['units']}"
    da_slice = da.sel(alt=slice(alt_start, alt_stop)).mean("alt", keep_attrs=True)
    da_slice.plot.contourf(cmap="Blues", **plot_kwargs)
    da_slice.plot.contour(colors="k", linewidths=0.5, **plot_kwargs)
    plt.title(title)
    plt.show()

## Plot interannual anomaly

In [ ]:
with xr.set_options(keep_attrs=True):
    da = ds["Q"].groupby("month") - ds_monthly["Q"]
da.attrs["long_name"] = f"Deseasonalized anomaly of {da.attrs['long_name']}"
for alt_start, alt_stop in zip(bins[:-1], bins[1:]):
    title = f"{da['alt'].attrs['long_name']}: {alt_start}-{alt_stop} {da['alt'].attrs['units']}"
    if alt_start < 4_000:
        vmax = 1.5
    elif alt_start > 4_000:
        vmax = 0.4
    else:
        vmax = 0.9
    da_slice = da.sel(alt=slice(alt_start, alt_stop)).mean("alt", keep_attrs=True)
    da_slice.plot.contourf(x="time", cmap="seismic", center=0, vmax=vmax, levels=101)
    plt.title(title)
    plt.show()

## Plot interannual anomaly %

In [ ]:
with xr.set_options(keep_attrs=True):
    da = 100 * (ds["Q"].groupby("time.month") / ds_monthly["Q"] - 1)
da.attrs |= {
    "long_name": f"Deseasonalized anomaly of {da.attrs['long_name']}",
    "units": "%",
}
for alt_start, alt_stop in zip(bins[:-1], bins[1:]):
    title = f"{da['alt'].attrs['long_name']}: {alt_start}-{alt_stop} {da['alt'].attrs['units']}"
    da_slice = da.sel(alt=slice(alt_start, alt_stop)).mean("alt", keep_attrs=True)
    da_slice.plot.contourf(
        x="time", cmap="seismic", center=0, vmax=50, levels=101, extend="both"
    )
    plt.title(title)
    plt.show()